In [8]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import  copy
import numpy as np

In [9]:
latent_dims = 2
num_epochs = 20
batch_size = 128
capacity = 64
learning_rate = 1e-3
#variational_beta = 0.05
variational_beta = 0.01
use_gpu = True



import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import optim

img_transform = transforms.Compose([
    transforms.ToTensor()
])
mnist_trainset = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
                   batch_size=batch_size, shuffle=True)

mnist_testset = test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
                  batch_size=batch_size, shuffle=True)

In [10]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        c = capacity
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=c, kernel_size=4, stride=2, padding=1) # out: c x 14 x 14
        self.conv2 = nn.Conv2d(in_channels=c, out_channels=c*2, kernel_size=4, stride=2, padding=1) # out: c x 7 x 7
        self.fc_mu = nn.Linear(in_features=c*2*7*7, out_features=latent_dims)
        self.fc_logvar = nn.Linear(in_features=c*2*7*7, out_features=latent_dims)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1) # flatten batch of multi-channel feature maps to a batch of feature vectors
        x_mu = self.fc_mu(x)
        x_logvar = self.fc_logvar(x)
        return x_mu, x_logvar


class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        c = capacity
        self.fc = nn.Linear(in_features=latent_dims, out_features=c*2*7*7)
        self.conv2 = nn.ConvTranspose2d(in_channels=c*2, out_channels=c, kernel_size=4, stride=2, padding=1)
        self.conv1 = nn.ConvTranspose2d(in_channels=c, out_channels=1, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), capacity*2, 7, 7) # unflatten batch of feature vectors to a batch of multi-channel feature maps
        x = F.relu(self.conv2(x))
        x = torch.sigmoid(self.conv1(x)) # last layer before output is sigmoid, since we are using BCE as reconstruction loss
        return x



class VariationalAutoencoder(nn.Module):
    def __init__(self):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        latent_mu, latent_logvar = self.encoder(x)
        latent = self.latent_sample(latent_mu, latent_logvar)
        x_recon = self.decoder(latent)
        return x_recon, latent_mu, latent_logvar

    def latent_sample(self, mu, logvar):
        if self.training:
            # the reparameterization trick
            std = logvar.mul(0.5).exp_()
            eps = torch.empty_like(std).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

def vae_loss(recon_x, x, mu, logvar,beta):

    recon_loss = F.binary_cross_entropy(recon_x.view(-1, 784), x.view(-1, 784), reduction='sum')
    kldivergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return recon_loss + variational_beta * kldivergence



vae = VariationalAutoencoder()

device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
vae = vae.to(device)

num_params = sum(p.numel() for p in vae.parameters() if p.requires_grad)
print('Number of parameters: %d' % num_params)
optimizer = optim.Adam(vae.parameters(), lr=7e-4)
rewind_state_dict = copy.deepcopy(vae.state_dict())



Number of parameters: 308357


## Pruning

In [11]:
from pruning import PruningConv2D, PruningLinear, PruningTool

conv = nn.Conv2d(in_channels=4, out_channels=4, kernel_size=4, stride=2, padding=1)

pr = PruningConv2D(conv)
pr.set_mask_locally(0.3)
for idx, channel in enumerate(conv.weight):
    pr(conv,None)

pr_stat = PruningTool()
pr_stat.stats_pruning(conv)

Model :
Sparsity in Layer : 25.00%
Global Sparsity : 25.00%


## Training :

In [12]:
step = 0
plt_loss = []
prstat = PruningTool()
first_training = True
pruning_amount = np.linspace(0, 0.99, 5)
final_pa = pruning_amount[-1]

for pa in pruning_amount:
    # local pruning
    for name,module in vae.named_modules():
        if isinstance(module, torch.nn.Linear):
            pr = PruningLinear(module)
            pr.set_mask_locally(pa)
            module.register_forward_pre_hook(pr)
        if isinstance(module, torch.nn.Conv2d):
            pr = PruningConv2D(module)
            pr.set_mask_locally(pa)
            module.register_forward_pre_hook(pr)

    for name, param in vae.named_parameters():
        param.data = rewind_state_dict[name].clone()

    epochs = 20 if pa == final_pa else 10
    for epoch in range(epochs):

        vae.train()
        train_loss = 0
        for batch_idx, (data, _) in enumerate(mnist_trainset):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = vae(data)
            beta = 1
            loss = vae_loss(recon_batch, data, mu, logvar,beta)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            step += 1

            if  first_training and batch_idx == 500 :
                rewind_state_dict = copy.deepcopy(vae.state_dict())
                first_training = False
                print('rewind state saved')
        if epoch ==0 :
            prstat.stats_pruning(vae,verbose=False)
        print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(mnist_trainset.dataset)))
    _, _, _ = vae(data)


Sparsity in Layer encoder.conv1: 0.00%
Sparsity in Layer encoder.conv2: 0.00%
Linear(in_features=6272, out_features=2, bias=True)
Sparsity in Layer encoder.fc_mu: 0.00%
Linear(in_features=6272, out_features=2, bias=True)
Sparsity in Layer encoder.fc_logvar: 0.00%
Linear(in_features=2, out_features=6272, bias=True)
Sparsity in Layer decoder.fc: 0.00%
Global Sparsity : 0.00%
====> Epoch: 0 Average loss: 187.1181
====> Epoch: 1 Average loss: 166.8918
====> Epoch: 2 Average loss: 161.2814
====> Epoch: 3 Average loss: 158.0389
====> Epoch: 4 Average loss: 156.0573
====> Epoch: 5 Average loss: 154.4578
====> Epoch: 6 Average loss: 152.9603
====> Epoch: 7 Average loss: 151.2438
====> Epoch: 8 Average loss: 149.7348
====> Epoch: 9 Average loss: 148.4107
Sparsity in Layer encoder.conv1: 0.00%
Sparsity in Layer encoder.conv2: 0.00%
Linear(in_features=6272, out_features=2, bias=True)
Sparsity in Layer encoder.fc_mu: 0.01%
Linear(in_features=6272, out_features=2, bias=True)
Sparsity in Layer encod

KeyboardInterrupt: 